#BMGT 347 Group project

###Group members: Samantha Wehmer, Leslie Kwok, Brian Garfunkel, Arthur Lin

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn import model_selection
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, add_dummy_feature

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#note: you need to copy the csv file to your Google drive and change the path below
data = pd.read_csv('/content/drive/My Drive/BMGT347/Final Project/characteristics_anom.csv')

#data = pd.read_csv('/content/drive/My Drive/BMGT 347 2023/Final project/characteristics_anom.csv')

data = data.set_index(['permno'])
data.head()

,date,re,size,value,prof,valprof,fscore,debtiss,repurch,nissa,...,rome,strev,ivol,betaarb,season,indrrev,ciss,price,age,shvol
permno,,,,,,,,,,,,,,,,,,,,,
10006,07/1972,0.028600,0.699925,0.649009,-0.773268,-0.441137,-1.057577,-1.191008,-0.443104,-2.053981,...,0.738251,0.169149,-0.253520,-0.597934,-0.762987,-0.549901,-1.220848,1.117585,1.213013,0.524622
10102,07/1972,0.039757,0.538023,1.406381,-0.604513,0.939781,0.432276,-1.191008,-0.443104,0.827987,...,1.094331,-0.677317,-0.658350,0.556359,-1.199386,-0.482659,-0.027322,-0.683487,1.213013,-0.411568
10145,07/1972,-0.062422,1.379123,1.073137,-0.641500,0.334379,0.432276,-1.191008,-0.443104,-0.480636,...,0.280873,-0.095248,0.235483,0.346043,0.557622,-0.108870,-0.970639,0.395970,1.213013,0.102147
10153,07/1972,-0.065600,0.186578,1.688356,-0.463498,1.385077,-0.339504,-1.191008,2.255521,-2.121410,...,0.937779,-1.223870,0.596222,0.776458,0.432121,-1.216393,0.660034,-1.280549,1.213013,0.177518
10161,07/1972,-0.048933,1.313967,0.667652,-1.170883,-0.898941,-1.057577,-1.191008,-0.443104,-0.195935,...,0.627744,0.944583,-1.145348,-0.350934,-0.680270,0.721773,-0.579507,0.427603,1.213013,-0.645615


In [ ]:
X_var_names = data.columns[2:]
y_var_name = data.columns[1]
print('X variables: ', X_var_names.values)
print('y variable: ', y_var_name)

X variables:  ['size' 'value' 'prof' 'valprof' 'fscore' 'debtiss' 'repurch' 'nissa'
 'accruals' 'growth' 'aturnover' 'gmargins' 'ep' 'cfp' 'noa' 'inv'
 'invcap' 'igrowth' 'sgrowth' 'lev' 'roaa' 'roea' 'sp' 'mom' 'indmom'
 'valmom' 'valmomprof' 'mom12' 'momrev' 'lrrev' 'valuem' 'nissm' 'roe'
 'rome' 'strev' 'ivol' 'betaarb' 'season' 'indrrev' 'ciss' 'price' 'age'
 'shvol']
y variable:  re


In [ ]:
# split the data into training and testing sets
data['date'] = pd.to_datetime(data['date'], format='%m/%Y')

# date of significant macroeconomic events
black_mon_drop = '1987-10'
fri_13_drop = '1989-10'
dotcom_drop_start = '2000-03'
dotcom_drop_end = '2002-10'
finCri_08_start = '2008-09'
finCri_08_end = '2009-03'

# Drop the rows where major economic or external events had caused a significant market disruption
data = data.loc[~((data['date'] == black_mon_drop) | (data['date'] == fri_13_drop) |
              ((data['date'] >= dotcom_drop_start) & (data['date'] <= dotcom_drop_end)) |
              ((data['date'] >= finCri_08_start) & (data['date'] <= finCri_08_end))), :]

print(data.describe())


                  re           size          value           prof  \
count  470413.000000  470413.000000  470413.000000  470413.000000   
mean        0.010736       0.200653       0.016364       0.260683   
std         0.104067       0.985308       0.962452       0.856234   
min        -0.767758      -1.731063      -1.730904      -1.730723   
25%        -0.046700      -0.628015      -0.797509      -0.462256   
50%         0.006229       0.300351      -0.011773       0.279444   
75%         0.061827       1.063942       0.845976       0.990544   
max         6.403007       1.731091       1.730978       1.731000   

             valprof         fscore        debtiss        repurch  \
count  470413.000000  470413.000000  470413.000000  470413.000000   
mean        0.104791       0.414742      -0.248762       0.239941   
std         0.963392       0.806702       0.959878       1.052684   
min        -1.750606      -1.974235      -1.646056      -1.394564   
25%        -0.699531      -0.1918

In [ ]:
# split the data into training and testing sets
train=data.loc[data['date']<"01/2005"]
test=data.loc[data['date']>="01/2005"]

X_train = train[X_var_names]
y_train = train[y_var_name]

X_test = test[X_var_names]
y_test = test[y_var_name]

print(test.head())

             date        re      size     value      prof   valprof    fscore  \
permno                                                                          
10026  2005-01-01 -0.017509 -1.475379  0.762284  1.378768  1.611330  0.415156   
10108  2005-01-01 -0.052430  1.269729 -0.414141  0.740092  0.391793  0.415156   
10225  2005-01-01  0.086506  1.383460 -0.544488  0.609340 -0.106919  0.415156   
10272  2005-01-01 -0.108743 -0.085687 -0.206246  0.559051  0.196918  0.415156   
10299  2005-01-01 -0.025336  1.414619 -1.460219 -0.143325 -1.624007  1.401380   

         debtiss   repurch     nissa  ...      rome     strev      ivol  \
permno                                ...                                 
10026   0.938875  1.259188 -0.267879  ...  1.244899  0.275382 -0.436955   
10108  -1.064640 -0.793816  0.204253  ...  1.216498  0.732896  0.058466   
10225  -1.064640  1.259188 -0.571277  ...  0.792065 -1.086589 -1.460108   
10272  -1.064640 -0.793816  0.455732  ... -0.593260 -1.69

# **OLS**

In [ ]:
# incorporated all 43 predictors within the formula
formula='re~'+("+").join(X_var_names.tolist())

In [ ]:
# fit ols model on training data
ols=smf.ols(formula=formula,data=train).fit()

#use model to make prediction on training data
ols_pred_train=ols.predict(X_train)

#evaluate performance of the model using R-Squared and RMSE metrics
print("OLS Training R-Squared:",r2_score(y_train,ols_pred_train))
print("OLS Training MSE:",mean_squared_error(y_train,ols_pred_train))

#use model to make prediction on testing data
ols_preds = ols.predict(X_test)

# evaluate the performance of the model using R-squared and RMSE metrics
print("OLS R-squared:", r2_score(y_test, ols_preds))
print("OLS MSE:", mean_squared_error(y_test, ols_preds))


OLS Training R-Squared: 0.006664878745711755
OLS Training MSE: 0.011410593989658937
OLS R-squared: -0.007437004283021853
OLS MSE: 0.009438148844424954


Although we were a little surprised by a negative r-squared, we rationalized it based on what we know about stock returns being clustered around 0. Given this, we believe its possible that a horizontal line at point (0,0) could be better than a regressed model.

# **Random Forest**

In our random forest model, we limit the number of estimators to 100 and the max depth to 7. These values allow for a shorter computational time and reduced likelihood of overfitting.

In [ ]:
# Random Forests
# fit Random Forest model on training data
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators = 100, max_depth = 7, random_state = 0).fit(X_train, y_train)

In [ ]:
# generate prediction for traning set data
rf_preds_train = forest.predict(X_train)
# evaluate the performance of the model in the training set
print("RF Training MSE:", mean_squared_error(y_train,rf_preds_train))
print("RF Training R-Squared:", r2_score(y_train,rf_preds_train))

# use the model to make predictions on the testing data
rf_preds = forest.predict(X_test)

# evaluate the performance of the model using R-squared and RMSE metrics
print("RF R-squared:", r2_score(y_test, rf_preds))
print("RF MSE:", mean_squared_error(y_test, rf_preds))

RF Training MSE: 0.011119430497573421
RF Training R-Squared: 0.03201175577750348
RF R-squared: -0.026667796443380798
RF MSE: 0.00961831204870872


In [ ]:
#feature importance
# print the feature importances of the model to see which variables are most predictive of excess returns
coeff_forest = pd.DataFrame()
feature_importances = forest.feature_importances_
coeff_forest['Feature importance'] = feature_importances
coeff_forest.index = X_var_names.values.tolist()
coeff_forest = coeff_forest.sort_values(by = 'Feature importance',ascending=False)
coeff_forest


,Feature importance
debtiss,0.311734
price,0.138037
repurch,0.086712
indrrev,0.069481
ivol,0.035372
rome,0.033450
mom12,0.026173
fscore,0.022938
roaa,0.019135
valprof,0.016877


The most important variables for predictability found were Debt Issuance (debtiss), Share Repurchases (repurch), and price (price).

# **Lasso**

In [ ]:
# fit Lasso regression model on training data using cross-validation to select regularization parameter
alphas = 10**np.linspace(3,-5,2000)*0.5

lasso = LassoCV(alphas=alphas, cv=10, random_state=30).fit(X_train, y_train)


In [ ]:
#use the model to make predictions on the training data
lasso_preds_train=lasso.predict(X_train)

#evaluate the performance of the training data using R-Squared and RMSE metrics
print("Lasso Training R-Squared:",r2_score(y_train,lasso_preds_train))
print("Lasso Training MSE:",mean_squared_error(y_train,lasso_preds_train))

Lasso Training R-Squared: 0.0063434370461997736
Lasso Training MSE: 0.011414286440117997


In [ ]:
# use the model to make predictions on the testing data
lasso_preds = lasso.predict(X_test)

# evaluate the performance of the model using R-squared and RMSE metrics
print("Lasso R-squared:", r2_score(y_test, lasso_preds))
print("Lasso MSE:", mean_squared_error(y_test, lasso_preds))

Lasso R-squared: -0.006138628263708368
Lasso MSE: 0.009425985040560083


In [ ]:
# print the coefficients of the model to see which variables are most predictive of excess returns
coeff_lasso = pd.DataFrame()
coefficients = lasso.coef_
coeff_lasso['Coefficient'] = coefficients
coeff_lasso['abs_coeff'] = abs(coeff_lasso['Coefficient'])
coeff_lasso.index = X_var_names.values.tolist()
coeff_lasso = coeff_lasso.sort_values(by = 'abs_coeff',ascending=False)
pd.DataFrame(coeff_lasso.iloc[:,0])

,Coefficient
indrrev,-0.005906
mom12,0.003287
rome,0.003027
age,-0.002580
season,0.002432
valuem,0.002055
betaarb,0.001631
mom,-0.001332
momrev,-0.001248
strev,0.001246


The more important variables for predictability found were Industry Relative Reversals (indrrev), Momentum (mom12), and Return on Market Equity.

Using MSE as the performance metrics, Random Forest has the lowest MSE score of 0.0111, thus the best in sample model. Lasso has the smallest MSE in the testing set at 0.00943, thus is the best out of smaple model.

# **CV for Best Method**

Cross-validation is used to determine which model is the most accurate. We used a cv value of 10 to achieve a relatively low computational time, and we used r-squared as the scorer to standardize and allow the cross-validation scores to be comparable.

In [ ]:
# cross validation within training sample to find best method

# Linear Regression
Linear_regression = LinearRegression()
print('Linear Cross Validation Score: ', cross_val_score(Linear_regression, X_train, y_train, cv = 10, scoring = 'r2' ).mean())

# Lasso
Lasso = LassoCV()
print('Lasso Cross Validation Score: ', cross_val_score(Lasso, X_train, y_train, cv = 10, scoring = 'r2').mean())

# Random Forest
Random_forest = RandomForestRegressor(n_estimators = 100, max_depth = 7, random_state = 0)
print('Random Forest Cross Validation Score: ', cross_val_score(Random_forest, X_train, y_train, cv = 10, scoring = 'r2').mean())

Linear Cross Validation Score:  0.0011668039510971395
Lasso Cross Validation Score:  0.001208940941319836
Random Forest Cross Validation Score:  -0.0025341346279090906


Lasso Model is determined to be the best model from the cross validation, with the highest CV score of 0.00121.